# DSA 2040 Practical Exam - Section 1: Data Warehousing

**Student Name:** Zakariya Shafi 
**Student ID:** 596 
**Date:** December 12, 2025

---

## Overview

This notebook demonstrates the complete **ETL (Extract, Transform, Load)** pipeline for building a Data Warehouse using a **Star Schema** design. The warehouse is designed to support OLAP queries for retail sales analysis.

### Objectives:
1. **Extract** data from the Online Retail CSV dataset
2. **Transform** the data (clean, calculate metrics, simulate current dates)
3. **Load** data into an SQLite Data Warehouse
4. **Visualize** insights from the warehouse

## 1. Import the required libraries
We'll use:
- `pandas` for data manipulation
- `sqlite3` for database operations

In [4]:
# ETL Process for Online Retail CSV

import pandas as pd
import sqlite3
from datetime import datetime

In [10]:
# -----------------------------
# Extract: Read CSV
# -----------------------------
def extract_data(filepath):
    # Read CSV (Online Retail)
    df = pd.read_csv("C:/Users/user/DSA2040_End_Sem/Online_Retail.csv", encoding='ISO-8859-1')

    print(f"Initial rows in CSV: {len(df)}")

    # Convert InvoiceDate to datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

    # Drop rows with missing CustomerID
    df = df.dropna(subset=['CustomerID'])
    print(f"Rows after dropping missing CustomerID: {len(df)}")
    
    return df


In [11]:
# -----------------------------
# Transform: Clean & Prepare
# -----------------------------
def transform_data(df):
    # Remove outliers (Quantity < 0 or UnitPrice <= 0)
    df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
    print(f"Rows after removing outliers: {len(df)}")

    # Create TotalSales column
    df['TotalSales'] = df['Quantity'] * df['UnitPrice']

    # Customer Dimension
    customer_dim = df.groupby('CustomerID').agg(
        TotalPurchases=('TotalSales', 'sum'),
        Country=('Country', 'first')
    ).reset_index()
    print(f"CustomerDim rows: {len(customer_dim)}")

    # Time Dimension
    df['Year'] = df['InvoiceDate'].dt.year
    df['Month'] = df['InvoiceDate'].dt.month
    df['Day'] = df['InvoiceDate'].dt.day
    time_dim = df[['InvoiceDate', 'Year', 'Month', 'Day']].drop_duplicates().reset_index(drop=True)
    print(f"TimeDim rows: {len(time_dim)}")

    # Filter for sales in last year (Aug 12, 2024 to Aug 12, 2025)
    last_year_start = datetime(2024, 8, 12)
    last_year_end = datetime(2025, 8, 12)
    sales_fact = df[(df['InvoiceDate'] >= last_year_start) & (df['InvoiceDate'] <= last_year_end)]
    print(f"SalesFact rows (last year): {len(sales_fact)}")

    return sales_fact, customer_dim, time_dim


In [12]:
# -----------------------------
# Load: Insert into SQLite
# -----------------------------
def load_data(sales_fact, customer_dim, time_dim, db_name='retail_dw.db'):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create tables
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS CustomerDim (
            CustomerID INTEGER PRIMARY KEY,
            TotalPurchases REAL,
            Country TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS TimeDim (
            InvoiceDate TEXT PRIMARY KEY,
            Year INTEGER,
            Month INTEGER,
            Day INTEGER
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS SalesFact (
            InvoiceNo TEXT,
            StockCode TEXT,
            Quantity INTEGER,
            UnitPrice REAL,
            TotalSales REAL,
            CustomerID INTEGER,
            InvoiceDate TEXT,
            FOREIGN KEY(CustomerID) REFERENCES CustomerDim(CustomerID),
            FOREIGN KEY(InvoiceDate) REFERENCES TimeDim(InvoiceDate)
        )
    ''')

    # Insert data
    customer_dim.to_sql('CustomerDim', conn, if_exists='replace', index=False)
    time_dim.to_sql('TimeDim', conn, if_exists='replace', index=False)
    sales_fact.to_sql('SalesFact', conn, if_exists='replace', index=False)

    conn.commit()
    conn.close()
    print(f"Data loaded into {db_name}")


In [15]:
# -----------------------------
# Full ETL Function
# -----------------------------
def etl_pipeline(filepath):
    df = extract_data(filepath)
    sales_fact, customer_dim, time_dim = transform_data(df)
    load_data(sales_fact, customer_dim, time_dim)

# -----------------------------
# Run ETL
# -----------------------------
if __name__ == "__main__":
     etl_pipeline("C:/Users/user/DSA2040_End_Sem/Online_Retail.csv") 

Initial rows in CSV: 541909
Rows after dropping missing CustomerID: 406829
Rows after removing outliers: 397884


C:\Users\user\AppData\Local\Temp\ipykernel_5600\2633493936.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalSales'] = df['Quantity'] * df['UnitPrice']


CustomerDim rows: 4338


C:\Users\user\AppData\Local\Temp\ipykernel_5600\2633493936.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['InvoiceDate'].dt.year
C:\Users\user\AppData\Local\Temp\ipykernel_5600\2633493936.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['InvoiceDate'].dt.month
C:\Users\user\AppData\Local\Temp\ipykernel_5600\2633493936.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

TimeDim rows: 17282
SalesFact rows (last year): 0
Data loaded into retail_dw.db
